# Light Curves Simulation for all SNe types

In [1]:
import snsim
import importlib
import matplotlib.pyplot as plt
import numpy as np

#secutiry check for numpy parallelization when using dask
def limit_numpy(nthreads=1):
    """ """
    import os
    threads = str(nthreads)
    print(f"threads {threads}")
    os.environ["NUMEXPR_NUM_THREADS"] = threads
    os.environ["OMP_NUM_THREADS"] = threads
    os.environ["OPENBLAS_NUM_THREADS"] = threads
    os.environ["MKL_NUM_THREADS"] = threads
    os.environ["VECLIB_MAXIMUM_THREADS"] = threads
limit_numpy(1)

import dask
import pandas as pd

threads 1


### we use dask to speed up the simulation

In [2]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 15.33 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37383,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 15.33 GiB
Comm: tcp://127.0.0.1:42055,Total threads: 4
Dashboard: http://127.0.0.1:41511/status,Memory: 3.83 GiB
Nanny: tcp://127.0.0.1:36695,


### init the survey

In [3]:
survey_conf = {
        'survey_file': './survey_file_example.csv',
        'sig_psf': 0.0,
        'sig_zp': 0.01,
        'zp': 25,
        'gain': 1,
        'ra_size': 7.295,
        'dec_size': 7.465,
        'noise_key': ['skynoise', 'skysigADU'], 
         }

### init the generator obejct for all SNe

In [4]:
snia_gen = {
    'M0': 'jla',
    'sigM': 0.1,
    'sct_model': 'G10',
    'force_n': 1000 ,
    "host_galaxy_noise" : False, #True if you want the noise given by host galaxy for that you need the galaxy mag in the
                                #observed bands + sersic radii a,b of the host galaxy in the host file
    'model_name': 'salt2',
    'alpha': 0.14,
    'beta': 2.9,
    'dist_x1': 'N21',
    'dist_c': [-0.055, 0.023, 0.150]}

# for simplicity we use same configuration for SN core collapse
sncc_gen = {'M0': -19,
            'sigM': [1.5,1.5],
            'rate': 'ztf20',
            'model_name': 'vin19_nocorr'
            }

cosmology = {'name':'planck18'}



cmb = {'v_cmb': 0,
       'l_cmb':0,
       'b_cmb':0}

mw_dust = {'model': 'CCM89',}

#distribution of peculiar velocities of SNe
vpec_dist = {'mean_vpec':0,
             'sig_vpec':300}


## configuration dictionary, input of simulation

In [5]:
config_dic = {'data': {'write_path': './',
                       'sim_name': 'Test_simulation_allSN',
                       'write_format': 'parquet'},
              'survey_config': survey_conf,
              'sim_par': {'randseed': 1234, # Optional
                          'z_range': [0.01, 0.3]},
              'snia_gen': snia_gen,
              'sniipl_gen': sncc_gen,
              'sniin_gen': sncc_gen,
              'sniib_gen': sncc_gen,
              'snib_gen': sncc_gen,
              'snic_gen': sncc_gen,
              'snic-bl_gen': sncc_gen,
              'cosmology': cosmology,
              'mw_dust': mw_dust,
              'vpec_dist': vpec_dist,
              'cmb': cmb,
              'dask': {'use': True,
                       'nworkers': 4}#parameters to use dask, nworkers depends on your machine
             }

In [6]:
#config dask
cluster.scale(config_dic['dask']['nworkers'])

## Run the simulation

In [7]:
%%time
#initialize simulator object
#first time a bit long sncosmo has to download all the Templates
sim = snsim.Simulator(config_dic)

CPU times: user 8.91 s, sys: 443 ms, total: 9.35 s
Wall time: 8.94 s


In [8]:
# -- Generate n base param
param_tmp = sim.generators[0].gen_basic_par(1000, 1234, 
                                            min_max_t=True)

In [9]:
epochs, params = sim.survey.get_observations(
            param_tmp,
            phase_cut=None,
            nep_cut=sim.nep_cut,
            IDmin=0,
            use_dask=sim.config['dask']['use'],
            npartitions=sim.config['dask']['nworkers'])

In [10]:
#run the simulation
sim.simulate()

      _______..__   __.         _______. __  .___  ___. 
     /       ||  \ |  |        /       ||  | |   \/   | 
    |   (----`|   \|  |       |   (----`|  | |  \  /  | 
     \   \    |  . `  |        \   \    |  | |  |\/|  | 
 .----)   |   |  |\   |    .----)   |   |  | |  |  |  | 
 |_______/    |__| \__|    |_______/    |__| |__|  |__| 
================================= Version : 0.5.0 ====== 
-----------------------------------------------------------

SIM NAME : Test_simulation_allSN
CONFIG FILE : No config file
SIM WRITE DIRECTORY : ./
SIMULATION RANDSEED : 1234

-----------------------------------------------------------

SURVEY FILE : ./survey_file_example.csv

First day in survey_file : 58000.00 MJD / 2017-09-04 00:00:00.000
Last day in survey_file : 58100.00 MJD / 2017-12-13 00:00:00.000

Survey effective duration is 100.00 days

Survey envelope area is 57.74 squared degrees (0.1 % of the sky)

No cut on survey file.

----------------------------------------------------------

### After the simulations

In [11]:
#the sim object has the attribute samples, which contain the data of all the object we simulated
sim.samples
#one sim.sample object for each generator 

In [12]:
sim.samples[0].header
#header with useful parameter of each simulation

{'seed': 1234,
 'seed_key': (0,),
 'obj_type': 'SNIa',
 'rate': 'lambda z: 3e-5',
 'model_name': ['salt2'],
 'model_version': ['T23'],
 'm_vp': 0,
 's_vp': 300,
 'mw_dust_model': 'CCM89',
 'mw_dust_rv': 3.1,
 'M0_band': 'bessell_b',
 'dist_x1': 'N21',
 'peak_c': -0.055,
 'dist_c': 'asym_gauss',
 'sig_c_low': 0.023,
 'sig_c_hi': 0.15,
 'cosmo': {'cosmod_name': 'planck18'}}

In [13]:
#meta attribute return the parameters of each object, same as sim_lcs.attrs
sim.samples[0].meta[0]

{'mu': 40.28117331936717,
 'zobs': 0.22321532306922487,
 'zCMB': 0.22321532306922487,
 'effects': ['mw_', 'G10_'],
 't0': 58034.90950342247,
 'x0': 7.025054615923283e-05,
 'x1': 1.1158189250371295,
 'c': -0.01899610101045291,
 'mw_ebv': 0.06874377744226459,
 'mw_r_v': 3.1,
 'G10_L0': 2157.3,
 'G10_F0': 0.0,
 'G10_F1': 0.000108,
 'G10_dL': 800.0,
 'G10_RndS': 998124575180,
 'ID': 0,
 'zcos': 0.2215437856652841,
 'como_dist': 929.2613557467457,
 'zpcmb': 0.0,
 'ra': 0.6614847077168625,
 'dec': 0.7580354265768443,
 'vpec': 410.23032726860123,
 'min_t': 58010.44519696108,
 'max_t': 58096.070269575925,
 '1_zobs': 1.2232153230692249,
 'model_name': 'salt2',
 'model_version': 'T23',
 'M0': -19.123830232811475,
 'coh_sct': -0.06105215635050651,
 'alpha': 0.14,
 'beta': 2.9,
 'mass_step': 0.0,
 'host_noise': False,
 'mb': 20.88498758776968}

In [14]:
#return the simulated lightcurves
sim.samples[0].sim_lcs

time   fluxtrue  fluxerrtrue        flux     fluxerr  \
ID  epochs                                                                 
0   0       58012.244898   0.305674   182.174132  -24.921924  182.241682   
    1       58014.285714   1.471778    81.018348  -23.627923   81.154968   
    2       58016.326531   2.900189    13.866707   -0.421725   13.777049   
    3       58018.367347   8.680345   430.407747 -607.830993  431.103212   
    4       58020.408163  13.210611   161.030858  -64.963515  161.191470   
...                  ...        ...          ...         ...         ...   
999 33      58091.836735   6.214402   663.915783  251.444285  664.100442   
    34      58093.877551   1.447700   751.928879  371.586957  752.174966   
    35      58095.918367   1.394434   158.296518  164.948255  158.812283   
    36      58097.959184   1.344201   680.281466  439.695361  680.603573   
    37      58100.000000   1.290721   402.707042 -108.155245  402.839703   

                  mag    magerr  zp zpsys  gain    skynoise  band  fieldID  \
ID  epochs                                                                   
0   0             NaN       NaN  25    ab     1  182.173293  ztfr        1   
    1             NaN       NaN  25    ab     1   81.009263  ztfr        1   
    2             NaN       NaN  25    ab     1   13.761709  ztfg        1   
    3             NaN       NaN  25    ab     1  430.397656  ztfr        1   
    4             NaN       NaN  25    ab     1  160.989787  ztfg        1   
...               ...       ...  ..   ...   ...         ...   ...      ...   
999 33      18.998896  2.867585  25    ab     1  663.911101  ztfr        1   
    34      18.574849  2.197772  25    ab     1  751.927917  ztfg        1   
    35      19.456631  1.045348  25    ab     1  158.292113  ztfg        1   
    36      18.392120  1.680609  25    ab     1  680.280478  ztfg        1   
    37            NaN       NaN  25    ab     1  402.705440  ztfg        1   

            fwhm_psf  sig_zp  
ID  epochs                    
0   0            0.5    0.01  
    1            0.5    0.01  
    2            0.5    0.01  
    3            0.5    0.01  
    4            0.5    0.01  
...              ...     ...  
999 33           0.5    0.01  
    34           0.5    0.01  
    35           0.5    0.01  
    36           0.5    0.01  
    37           0.5    0.01  

[31597 rows x 15 columns]

## SNIA LC SIM with HOST Galaxy Noise

In [15]:
%%time

snia_gen = {
    'M0': 'jla',
    'sigM': 0.1,
    'sct_model': 'G10',
    'force_n': 2000 ,

    'model_name': 'salt2',
    'alpha': 0.14,
    'beta': 2.9,
    'dist_x1': 'N21',
    'dist_c': [-0.055, 0.023, 0.150]}

host_conf ={
    'host_file': './test_mock_nbody_new.parquet',
    'key_dic': {'vpec_true': 'vpec'}, #attention this depend on your mock file 
    'distrib': 'mass_sfr',
    "host_noise" : True, #True if you want the noise given by host galaxy for that you need the galaxy mag in the
                                #observed bands + sersic radii a,b of the host galaxy in the host file}

config_dic = {'data': {'write_path': './',
                       'sim_name': 'Test_simulation_SNIa_galaxynoise',
                       'write_format': 'parquet'},
              'survey_config': survey_conf,
              'sim_par': {'randseed': 1234, # Optional
                          'z_range': [0.01, 0.3]},
              'snia_gen': snia_gen,
              'cosmology': cosmology,
              'host':host_conf,
              'cmb': cmb,
              'dask': {'use': True,
                       'nworkers': 4}#parameters to use dask, nworkers depends on your machine
             }

sim = snsim.Simulator(config_dic)
sim.simulate()

ValueError: Set vpec_dist xor hosts

In [ ]:
host = snsim.SnHost(host_conf)

In [ ]:
sim.samples[0].sim_lcs.columns

In [ ]:
sim.samples[0].sim_lcs.galaxy_noise